#**스마트폰 센서 데이터 기반 모션 분류**
# 단계3 : 단계별 모델링


## 0.미션

단계별로 나눠서 모델링을 수행하고자 합니다.  

* 단계1 : 정적(0), 동적(1) 행동 분류 모델 생성
* 단계2 : 세부 동작에 대한 분류모델 생성
    * 단계1 모델에서 0으로 예측 -> 정적 행동 3가지 분류 모델링
    * 단계1 모델에서 1으로 예측 -> 동적 행동 3가지 분류 모델링 
* 모델 통합
    * 두 단계 모델을 통합하고, 새로운 데이터에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
* 성능 비교
    * 기본 모델링의 성능과 비교
    * 모든 모델링은 [다양한 알고리즘 + 성능 튜닝]을 수행해야 합니다.


## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용

 <br/>  

* 세부 요구사항
    - data01_train.csv 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - data01_test.csv 를 불러와 'new_data' 이름으로 저장합니다.


In [87]:
data = pd.read_csv("/content/drive/MyDrive/2023.04.12_미니프로젝트5차_3_5일차 실습자료/data01_train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/미프5/test_dataset.csv")

In [3]:
fi = pd.read_csv("/content/drive/MyDrive/2023.04.12_미니프로젝트5차_3_5일차 실습자료/fi.csv")

## 2.데이터 전처리

* 세부 요구사항
    - Label 추가 : data 에 Activity_dynamic 를 추가합니다. Activity_dynamic은 과제1에서 is_dynamic과 동일한 값입니다.
    - x와 y1, y2로 분할하시오.
        * y1 : Activity
        * y2 : Activity_dynamic
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [4]:
data['Activity_dynamic'] = data['Activity']

In [5]:
data.drop('subject', axis=1, inplace=True)

In [6]:
label_dict = {
    'LAYING': 0,
    'STANDING': 0,
    'SITTING': 0,
    'WALKING': 1,
    'WALKING_UPSTAIRS': 1,
    'WALKING_DOWNSTAIRS': 1
}
data_df = data.replace({'Activity_dynamic':label_dict}).copy()

In [88]:
data_df

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity,Activity_dynamic
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,STANDING,0
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,LAYING,0
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,STANDING,0
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,-0.408956,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388,WALKING,1
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.563437,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758,WALKING_DOWNSTAIRS,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5876,0.277194,-0.012389,-0.131974,-0.994046,-0.940578,-0.917337,-0.994261,-0.932830,-0.908088,-0.936219,...,-0.690363,-0.034888,-0.261437,-0.391477,-0.877612,-0.912365,0.114009,0.080146,SITTING,0
5877,0.191568,0.013328,-0.105174,-0.126969,-0.121729,-0.327480,-0.192523,-0.109923,-0.295286,0.078644,...,-0.879215,0.721718,0.623151,0.866858,-0.445660,-0.690278,0.303194,-0.044188,WALKING_UPSTAIRS,1
5878,0.267981,-0.018348,-0.107440,-0.991303,-0.989881,-0.990313,-0.992386,-0.988852,-0.991237,-0.936099,...,-0.886851,0.060173,0.228739,0.684400,-0.216665,0.620363,-0.437247,-0.571840,LAYING,0
5879,0.212787,-0.048130,-0.121001,-0.041373,0.052449,-0.585361,-0.100714,0.023353,-0.554707,0.219814,...,-0.053556,0.260880,0.551742,-0.943773,-0.862899,-0.718009,0.292856,0.024920,WALKING_UPSTAIRS,1


## **3.단계별 모델링**

![](https://github.com/DA4BAM/image/blob/main/step%20by%20step.png?raw=true)

### (1) 단계1 : 정적/동적 행동 분류 모델

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)과 동적 행동(동적 : Walking, Walking-Up, Walking-Down)을 구분하는 모델 생성.
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [89]:
two = list(fi.sort_values('fi(6behaviour)', ascending=False)['feature_name'].head(20))
two

['tGravityAcc-mean()-Y',
 'angle(X,gravityMean)',
 'tGravityAcc-min()-X',
 'tGravityAcc-max()-Y',
 'tGravityAcc-mean()-X',
 'tGravityAcc-min()-Y',
 'tGravityAcc-energy()-X',
 'angle(Y,gravityMean)',
 'tGravityAcc-max()-X',
 'fBodyAccJerk-bandsEnergy()-1,8',
 'tGravityAcc-energy()-Y',
 'tGravityAcc-arCoeff()-Z,1',
 'fBodyAccJerk-bandsEnergy()-1,16',
 'tGravityAcc-max()-Z',
 'tBodyAcc-max()-X',
 'angle(Z,gravityMean)',
 'fBodyAccMag-energy()',
 'fBodyAcc-bandsEnergy()-1,16',
 'fBodyAccJerk-std()-X',
 'tGravityAcc-arCoeff()-Y,3']

In [92]:
target = 'Activity_dynamic'
x_t = data_df[two]
y = data_df.loc[:,'Activity']
# y1 = data_df.loc[:,'Activity']
# y2 = data_df.loc[:,target]

# from sklearn.model_selection import train_test_split
# x_train_t ,x_val_t, y_train_t, y_val_t = train_test_split(x_t, y2, test_size = 0.2,  random_state=1)

In [93]:
x_t

,tGravityAcc-mean()-Y,"angle(X,gravityMean)",tGravityAcc-min()-X,tGravityAcc-max()-Y,tGravityAcc-mean()-X,tGravityAcc-min()-Y,tGravityAcc-energy()-X,"angle(Y,gravityMean)",tGravityAcc-max()-X,"fBodyAccJerk-bandsEnergy()-1,8",tGravityAcc-energy()-Y,"tGravityAcc-arCoeff()-Z,1","fBodyAccJerk-bandsEnergy()-1,16",tGravityAcc-max()-Z,tBodyAcc-max()-X,"angle(Z,gravityMean)",fBodyAccMag-energy(),"fBodyAcc-bandsEnergy()-1,16",fBodyAccJerk-std()-X,"tGravityAcc-arCoeff()-Y,3"
0,-0.358842,-0.601120,0.891969,-0.370494,0.875254,-0.334921,0.673768,0.331298,0.805627,-0.999955,-0.775936,-0.386458,-0.999952,-0.207900,-0.929747,0.165163,-0.999653,-0.999906,-0.987847,-0.272900
1,0.924656,0.345205,-0.095343,0.873312,-0.134711,0.937432,-0.893830,-0.769186,-0.200892,-0.999961,0.714508,0.021547,-0.999946,0.227824,-0.937337,-0.147944,-0.999866,-0.999902,-0.992979,0.028043
2,-0.174095,-0.833564,0.985606,-0.195671,0.965965,-0.145801,0.906252,0.202434,0.891615,-0.999996,-0.953043,0.075337,-0.999978,0.069356,-0.942584,-0.032755,-0.999897,-0.999991,-0.994513,0.152300
3,-0.297703,-0.695819,0.943954,-0.309430,0.927343,-0.275699,0.805419,0.287154,0.858160,-0.769630,-0.849096,-0.352702,-0.726715,-0.127354,0.008288,0.111388,-0.850144,-0.705427,-0.229423,-0.318939
4,-0.260540,-0.705029,0.901943,-0.236084,0.901125,-0.263482,0.738738,0.264952,0.858531,-0.510367,-0.885215,-0.769572,-0.596334,-0.080139,0.584131,0.137758,-0.330888,-0.373369,-0.212695,-0.594359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5876,-0.044093,-0.912365,0.992398,-0.063552,0.973223,-0.027374,0.925506,0.114009,0.899036,-0.999979,-0.998616,-0.649029,-0.999963,-0.076862,-0.936219,0.080146,-0.998690,-0.999964,-0.992318,-0.455731
5877,-0.317517,-0.690278,0.921001,-0.325357,0.910932,-0.291398,0.763412,0.303194,0.843681,-0.756097,-0.826961,-0.369010,-0.695006,0.091460,0.078644,-0.044188,-0.719266,-0.591533,-0.263827,-0.369787
5878,0.684910,0.620363,-0.465011,0.639925,-0.514220,0.700942,-0.998561,-0.437247,-0.579367,-0.999936,-0.046641,-0.122467,-0.999936,0.741853,-0.936099,-0.571840,-0.999891,-0.999946,-0.986582,-0.003316
5879,-0.302780,-0.718009,0.932387,-0.314572,0.921553,-0.293450,0.790552,0.292856,0.852565,-0.576003,-0.843467,-0.699714,-0.698184,0.013870,0.219814,0.024920,-0.690162,-0.520729,-0.200845,-0.555938


In [94]:
target = 'Activity_dynamic'
x = data_df.drop([target, 'Activity'], axis=1)
y = data_df.loc[:,'Activity']
# y1 = data_df.loc[:,'Activity']
# y2 = data_df.loc[:,target]

# from sklearn.model_selection import train_test_split
# x_train ,x_val, y_train, y_val = train_test_split(x, y2, test_size = 0.2,  random_state=1)

In [98]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(y)

In [95]:
x

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.032207,-0.487737,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.246705,-0.237820,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,0.388765,-0.535287,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,0.050888,-0.004012,-0.408956,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.013902,-0.157832,-0.563437,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5876,0.277194,-0.012389,-0.131974,-0.994046,-0.940578,-0.917337,-0.994261,-0.932830,-0.908088,-0.936219,...,0.052914,-0.305772,-0.690363,-0.034888,-0.261437,-0.391477,-0.877612,-0.912365,0.114009,0.080146
5877,0.191568,0.013328,-0.105174,-0.126969,-0.121729,-0.327480,-0.192523,-0.109923,-0.295286,0.078644,...,0.325461,-0.622617,-0.879215,0.721718,0.623151,0.866858,-0.445660,-0.690278,0.303194,-0.044188
5878,0.267981,-0.018348,-0.107440,-0.991303,-0.989881,-0.990313,-0.992386,-0.988852,-0.991237,-0.936099,...,0.362439,-0.597610,-0.886851,0.060173,0.228739,0.684400,-0.216665,0.620363,-0.437247,-0.571840
5879,0.212787,-0.048130,-0.121001,-0.041373,0.052449,-0.585361,-0.100714,0.023353,-0.554707,0.219814,...,0.031915,0.196921,-0.053556,0.260880,0.551742,-0.943773,-0.862899,-0.718009,0.292856,0.024920


#### 1) 알고리즘1 : 

In [99]:
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import *
model = XGBClassifier(n_estimators=400)
model.fit(x, y)
# 예측하기
XGBC_pred = model.predict(test_data)

# result={}
# # 평가하기
# print(confusion_matrix(y_val_t, XGBC_pred))
# print(classification_report(y_val_t, XGBC_pred))
# result['XGBClassifier'] = accuracy_score(y_val_t, XGBC_pred)

ValueError: ignored

In [ ]:
XGBC_pred

In [31]:
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import *
model = XGBClassifier(n_estimators=400)
model.fit(x_train, y_train)
# 예측하기
XGBC_pred = model.predict(x_val)

result={}
# 평가하기
print(confusion_matrix(y_val, XGBC_pred))
print(classification_report(y_val, XGBC_pred))
result['XGBClassifier'] = accuracy_score(y_val, XGBC_pred)

[[214   0   0]
 [  0 163   0]
 [  0   1 152]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       214
           1       0.99      1.00      1.00       163
           2       1.00      0.99      1.00       153

    accuracy                           1.00       530
   macro avg       1.00      1.00      1.00       530
weighted avg       1.00      1.00      1.00       530



#### 2) 알고리즘2 : 

### (2) 단계2-1 : 정적 동작 세부 분류

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)인 데이터 추출
    * Laying, Sitting, Standing 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [14]:
inactive_df = data_df[data_df['Activity_dynamic']==0]

In [60]:
index1 = list(inactive_df.index)

In [61]:
target = 'Activity_dynamic'
x = inactive_df.drop([target, 'Activity'], axis=1)
y = inactive_df.loc[:,'Activity']

from sklearn.model_selection import train_test_split
x_train ,x_val, y_train, y_val = train_test_split(x, y, test_size = 0.2, random_state=1)

In [62]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y_train = encoder.fit_transform(y_train)
y_val = encoder.fit_transform(y_val)

In [63]:
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import *
model = XGBClassifier(n_estimators=400)
model.fit(x_train, y_train)
# 예측하기
activity_pred = model.predict(x_val)

result={}
# 평가하기
print(confusion_matrix(y_val, activity_pred))
print(classification_report(y_val, activity_pred))
result['XGBClassifier'] = accuracy_score(y_val, activity_pred)

[[204   0   0]
 [  0 209  11]
 [  0   5 218]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       204
           1       0.98      0.95      0.96       220
           2       0.95      0.98      0.96       223

    accuracy                           0.98       647
   macro avg       0.98      0.98      0.98       647
weighted avg       0.98      0.98      0.98       647



### (3) 단계2-2 : 동적 동작 세부 분류

* 세부 요구사항
    * 동동적 행동(Walking, Walking Upstairs, Walking Downstairs)인 데이터 추출
    * Walking, Walking Upstairs, Walking Downstairs 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [64]:
active_df = data_df[data_df['Activity_dynamic']==1]

In [65]:
index2 = list(active_df.index)

In [66]:
target = 'Activity_dynamic'
x = active_df.drop([target, 'Activity'], axis=1)
y = active_df.loc[:,'Activity']

from sklearn.model_selection import train_test_split
x_train ,x_val, y_train, y_val = train_test_split(x, y, test_size = 0.2, random_state=1)

In [67]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y_train = encoder.fit_transform(y_train)
y_val = encoder.fit_transform(y_val)

In [68]:
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import *
model = XGBClassifier(n_estimators=400)
model.fit(x_train, y_train)
# 예측하기
inactive_pred = model.predict(x_val)

result={}
# 평가하기
print(confusion_matrix(y_val, inactive_pred))
print(classification_report(y_val, inactive_pred))
result['XGBClassifier'] = accuracy_score(y_val, inactive_pred)

[[214   0   0]
 [  0 163   0]
 [  0   1 152]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       214
           1       0.99      1.00      1.00       163
           2       1.00      0.99      1.00       153

    accuracy                           1.00       530
   macro avg       1.00      1.00      1.00       530
weighted avg       1.00      1.00      1.00       530



### (4) 분류 모델 합치기


* 세부 요구사항
    * 두 단계 모델을 통합하고, 새로운 데이터(test)에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
    * 데이터 파이프라인 구축 : test데이터가 로딩되어 전처리 과정을 거치고, 예측 및 성능 평가 수행

![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

In [69]:
activity_pred

array([0, 2, 1, 2, 1, 1, 2, 1, 2, 2, 0, 0, 2, 0, 0, 1, 2, 1, 0, 1, 0, 2,
       2, 2, 2, 1, 2, 0, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2,
       2, 2, 0, 0, 1, 0, 1, 1, 1, 0, 2, 2, 0, 2, 2, 1, 2, 1, 1, 1, 1, 2,
       1, 2, 2, 0, 2, 2, 2, 0, 1, 2, 0, 0, 2, 0, 2, 0, 2, 1, 1, 1, 0, 0,
       0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 0, 1, 1, 0, 2, 2, 1, 1, 2, 0, 2,
       0, 1, 0, 2, 2, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 2, 0, 1, 1, 2, 2, 2, 0, 0, 1, 2, 0, 2,
       0, 0, 0, 2, 1, 1, 2, 2, 0, 1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1,
       2, 0, 1, 0, 0, 2, 2, 2, 0, 2, 0, 0, 0, 1, 2, 2, 1, 1, 0, 2, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 2,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 2, 2, 2, 1, 0, 0, 2, 1, 0, 0, 1, 2, 2,
       1, 1, 1, 1, 2, 1, 0, 0, 1, 0, 1, 2, 0, 1, 0, 2, 1, 0, 2, 2, 1, 0,
       2, 1, 2, 2, 0, 1, 2, 2, 0, 1, 1, 1, 1, 0, 1, 2, 1, 1, 0, 0, 2, 2,
       2, 2, 1, 2, 2, 0, 0, 2, 2, 0, 2, 0, 1, 0, 0,

In [70]:
inactive_pred

array([0, 1, 0, 0, 0, 2, 2, 2, 1, 2, 2, 0, 1, 2, 0, 0, 0, 0, 2, 2, 1, 2,
       1, 0, 0, 2, 2, 0, 1, 0, 0, 2, 1, 0, 0, 2, 1, 1, 0, 1, 2, 0, 0, 0,
       0, 2, 0, 0, 0, 2, 1, 0, 2, 1, 2, 2, 0, 2, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 2, 0, 2, 2, 0, 0, 1, 2, 1, 2, 1, 0, 1,
       0, 2, 2, 2, 0, 1, 1, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 2, 2, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1,
       1, 0, 0, 1, 2, 1, 2, 0, 0, 2, 2, 2, 0, 2, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 2, 2, 2, 0, 1, 2, 2, 2, 1, 1, 1, 2, 0, 0, 0, 2, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 2, 2, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 2, 2, 0, 0, 0, 2,
       0, 2, 2, 1, 0, 0, 2, 1, 0, 0, 1, 1, 0, 2, 0, 0, 0, 2, 2, 2, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 2, 0, 1, 1, 1, 1, 2, 0, 0, 0, 0, 2, 2,
       2, 0, 1, 2, 0, 1, 0, 2, 1, 0, 0, 0, 2, 0, 1, 0, 1, 2, 0, 2, 1, 2,
       0, 0, 0, 1, 2, 2, 2, 0, 1, 0, 0, 0, 2, 0, 1, 0, 1, 2, 2, 0, 0, 2,
       0, 1, 0, 2, 0, 2, 2, 1, 0, 0, 2, 2, 0, 1, 1,

In [78]:
len(data_df)

5881

In [79]:
len(activity_pred)

647

In [80]:
len(inactive_pred)

530

In [77]:
arr = [0] * len(data_df)

count =0
for index in index1:
    arr[index] = activity_pred[count][0]
    count += 1

IndexError: ignored

In [ ]:
for index in index2:
    arr[index] = inactive_pred[count][0]
    count += 1

#### 1) 함수 만들기